In [ ]:
#default_exp ranker

In [ ]:
#hide
from nbdev.showdoc import *

# Ranker

Takes a query and an index and finds the nearest neighbors or most similar scores. Ideally this is just a simple Annoy `get_nns_by_vector`, or in the simple case a similarity score across all the vectors.

In [ ]:
import torch


from pathlib import Path

from memery.loader import treemap_loader, db_loader
from memery.encoder import text_encoder

In [ ]:
treemap = treemap_loader(Path('images/memery.ann'))

In [ ]:
treemap.get_n_items()

79

In [ ]:
#export
def ranker(query_vec, treemap):
    nn_indexes = treemap.get_nns_by_vector(query_vec[0], treemap.get_n_items())
    return(nn_indexes)

In [ ]:
#export
def nns_to_files(db, indexes):
    return([[v['fpath'] for k,v in db.items() if v['index'] == ind][0] for ind in indexes])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
db = db_loader(Path('images/memery.pt'), device)

In [ ]:
query = 'dog'

In [ ]:
query_vec = text_encoder(query, device)
indexes = ranker(query_vec, treemap)
ranked_files = nns_to_files(db, indexes)

In [ ]:
ranked_files

['images/Wholesome-Meme-8.jpg',
 'images/Wholesome-Meme-5.jpg',
 'images/Wholesome-Meme-35.jpg',
 'images/Wholesome-Meme-67.png',
 'images/embarassed-dog-on-bed-SA2BDZW.jpg',
 'images/Wholesome-Meme-72.jpg',
 'images/braydon-anderson-wOHH-NUTvVc-unsplash-min.jpg',
 'images/cute-dog-with-cupcake-P9E2YL5-min.jpg',
 'images/Wholesome-Meme-3.jpg',
 'images/Wholesome-Meme-18.jpg',
 'images/Wholesome-Meme-29.jpg',
 'images/Wholesome-Meme-13.jpg',
 'images/Wholesome-Meme-14.jpg',
 'images/Wholesome-Meme-71.jpg',
 'images/Wholesome-Meme-44.png',
 'images/Wholesome-Meme-63.jpg',
 'images/Wholesome-Meme-15.jpg',
 'images/Wholesome-Meme-17.jpg',
 'images/Wholesome-Meme-98.jpg',
 'images/stonks-meme.jpg',
 'images/Wholesome-Meme-39.jpg',
 'images/Wholesome-Meme-68.jpg',
 'images/Wholesome-Meme-45.jpg',
 'images/Wholesome-Meme-36.jpg',
 'images/Wholesome-Meme-22.jpg',
 'images/Wholesome-Meme-77.jpg',
 'images/Wholesome-Meme-61.png',
 'images/Wholesome-Meme-84.jpg',
 'images/Wholesome-Meme-25.jpg',
